Metaclasse
- Cria classe de forma dinâmica.
- Pode ser útil na criação de frameworks onde as classes precisam ter um comportamento específico.

In [1]:
class MeuMeta(type):
    def __new__(cls, nome, bases, dct):
        dct['novo_atributo'] = 'Valor adicionado pela metaclasse'
        return super().__new__(cls, nome, bases, dct)

In [2]:
class MinhaClasse(metaclass=MeuMeta):
    pass

In [3]:
obj = MinhaClasse()
obj.novo_atributo

'Valor adicionado pela metaclasse'

In [20]:
class ValidadorMeta(type):
    def __new__(cls, nome, bases, dct):
        validacoes = dct.get('validacoes', {})
        
        for attr, tipo in validacoes.items():
            if not callable(tipo):
                raise TypeError(f"A validação de '{attr}' deve ser uma função")
            # Adiciona a função de validação ao dicionário de atributos
            def valida_func(self, value, attr=attr, tipo=tipo):
                if not isinstance(value, tipo):
                    raise ValueError(f"O atributo '{attr}' deve ser do tipo {tipo.__name__}")
                self.__dict__[attr] = value
                
            # Renomeia a função para evitar problemas de escopo
            valida_func.__name__ = f"set_{attr}"
            dct[f"set_{attr}"] = valida_func

        return super().__new__(cls, nome, bases, dct)

In [23]:
class Usuario(metaclass=ValidadorMeta):
    validacoes = {
        'nome': str,
        'idade': int
    }

    def __init__(self, nome, idade):
        self.set_nome(nome)
        self.set_idade(idade)


In [24]:
try:
    user = Usuario("Alice", 30)
    print(f"Nome: {user.nome} - Idade: {user.idade}")
    
    # Tentativa de passar valor inválido
    user.set_idade("trinta")
except ValueError as e:
    print(e)

Nome: Alice - Idade: 30
O atributo 'idade' deve ser do tipo int
